- [北京2012年721](https://zh.wikipedia.org/zh-cn/2012%E5%B9%B4%E5%8C%97%E4%BA%AC%E7%89%B9%E5%A4%A7%E6%9A%B4%E9%9B%A8)：区站号 54416
- [福建武夷山2010](https://www.chinanews.com.cn/gn/news/2010/06-24/2360481.shtml)：区站号 58730
- [江西景德镇](https://m.huanqiu.com/article/9CaKrnJwEZx)：区站号 58527

<!-- 你抽时间还是再尝试一下百年一遇，当然这个和论文就没关系了，因为我还是想让你继续把这个问题往深入探讨探讨，算是我们一个小课题研究吧。一个就是找2个百年一遇的地方，不要太多，先只要两个，可以把他们的灾区可以化成50公里范围之内，近灾区50~100公里，远灾区100~200公里或者更大一些范围，也就是非灾区也不要扩大到全国。这个时候来去看一下他们的情况，数据处理的方式都和现在20年一遇一样，就是不能出现一会儿又是灾区，一会儿又是近灾区。看看这个时候结果怎么样 -->

In [1]:
import pandas as pd
import clickhouse_connect
from geopy.distance import distance
import statsmodels.formula.api as smf
from numpy import log
import logging
from stargazer.stargazer import Stargazer
logging.basicConfig(level=logging.INFO)

def get_distance(lat1, lon1, lat2, lon2):
    return distance((lat1, lon1), (lat2, lon2)).km

distance_min, distance_middle, distance_max = 50, 100, 200
client = clickhouse_connect.get_client(host='localhost', database='thesis')

events=client.query_df("select * from history_raining_100 where `区站号` in (58730, 54416) and `年`<2015 and `年`>1995  order by `20-20时累计降水量` desc").drop_duplicates(subset=['区站号'])
events

,locations,经度,纬度,record_date,区站号,20-20时累计降水量,年,maxraining
0,wsgujwgw1742,118.02,27.46,2010-06-20,58730,1684,2010,1635.976055
1,wx4ug7xk8z93,116.52,40.23,2012-07-21,54416,1576,2012,1529.771342


In [2]:
row = events.iloc[1]

def fetch_insurances(row):
    logging.info(f"fetching insurances for {row["区站号"]}")
    insurances = client.query_df(f"select greatCircleDistance(`标的经度`, `标的纬度`, {row["经度"]}, {row["纬度"]})/1000 as distance, * from bases where distance<{distance_max} order by distance asc")
    insurances["near"]=(insurances["distance"]<distance_min).astype(int)
    insurances["middle"]=insurances["distance"].between(distance_min, distance_middle).astype(int)
    insurances["far"]=insurances["distance"].between(distance_middle, distance_max).astype(int)
    return insurances

insurances=fetch_insurances(row)
insurances.groupby(["near", "middle", "far"]).count()


INFO:root:fetching insurances for 54416


distance  locations    保险起期    保险止期    标的经度    标的纬度   上年保单号  \
near middle far                                                                
0    0      1      106726     106726  106726  106726  106726  106726  106726   
     1      0       29333      29333   29333   29333   29333   29333   29333   
1    0      0      186609     186609  186609  186609  186609  186609  186609   

                    保单号    保险金额    保费合计  ...    复核保费  保险金额.1  保险费率合计  保险费率分解  \
near middle far                          ...                                   
0    0      1    106726  106726  106726  ...  106726  106726  106726  106726   
     1      0     29333   29333   29333  ...   29333   29333   29333   29333   
1    0      0    186609  186609  186609  ...  186609  186609  186609  186609   

                   主险保额    主险保费    主险费率  复核保费合计   附加险保费     总保费  
near middle far                                                  
0    0      1    106726  106726  106726  106726  106726  106726  
     1      0     29333   29333   29333   29333   29333   29333  
1    0      0    186609  186609  186609  186609  186609  186609  

[3 rows x 34 columns]

In [3]:
def fetch_extra_rainings(row):
    logging.info(f"fetching extra rainings for {row['区站号']}")
    rainings = client.query_df(
    f"select greatCircleDistance(`经度`, `纬度`, {row['经度']}, {row['纬度']})/1000 as distance,* from history_raining_100 where record_date between '{(row['record_date']-pd.Timedelta(days=5*365)).strftime('%Y-%m-%d')}' and '{row['record_date'].strftime('%Y-%m-%d')}' and distance<{distance_middle} and distance>0"
)

    return rainings

rainings = fetch_extra_rainings(row)
rainings

INFO:root:fetching extra rainings for 54416


""


In [4]:
def delete_extras(rainings, insurances):
    if rainings.empty:
        logging.info("no extra rainings")
        return insurances
    for _,rain in rainings.iterrows():
        logging.info(f"deleting extras for {rain['区站号']}")
        insurances["dist"]=insurances.apply(lambda x: get_distance(rain["纬度"], rain["经度"], x["标的纬度"], x["标的经度"]), axis=1)
        insurances["dist_flag"]=insurances["dist"].apply(lambda x: 0 if x>distance_middle else 1)
        insurances = insurances[insurances["dist_flag"]==0].copy()
    return insurances.drop(columns=["dist", "dist_flag"])

insurances=delete_extras(rainings, insurances)
insurances

INFO:root:no extra rainings


,distance,locations,保险起期,保险止期,标的经度,标的纬度,上年保单号,保单号,保险金额,保费合计,...,保险费率分解,主险保额,主险保费,主险费率,复核保费合计,附加险保费,总保费,near,middle,far
0,1.030419,wx4ugd1c433v,2004-07-23,2005-07-23,116.523499,40.221127,,04FCB0F603586712E05400144F67CA43,296000.0,260.000000,...,,296000.0,260.000000,0.000878,260.000000,0.0,260.000000,1,0,0
1,1.060964,wx4ug9fpfetr,2005-01-26,2006-01-26,116.523857,40.220924,,04FCB12BDC1A6712E05400144F67CA43,100000.0,49.000000,...,,100000.0,49.000000,0.000490,49.000000,0.0,49.000000,1,0,0
2,1.060964,wx4ug9fpfetr,2005-02-02,2006-02-02,116.523857,40.220924,,04FCB12BFC8C6712E05400144F67CA43,60000.0,29.400000,...,,60000.0,29.400000,0.000490,29.400000,0.0,29.400000,1,0,0
3,1.060964,wx4ug9fpfetr,2005-02-02,2006-02-02,116.523857,40.220924,,04FCB12BFCEC6712E05400144F67CA43,65000.0,31.850000,...,,65000.0,31.850000,0.000490,31.850000,0.0,31.850000,1,0,0
4,1.380379,wx4ugmztp210,2000-09-16,2003-11-16,116.520622,40.242405,,04FCB0F279556712E05400144F67CA43,53190.0,344.670013,...,,53190.0,344.670013,0.006480,344.670013,0.0,344.670013,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322663,199.894292,wxh2u21jsu9e,2012-05-23,2015-05-22,118.664909,39.507805,,04FCB0F00C7F6712E05400144F67CA43,300000.0,630.000000,...,(万分之)第一年8.83，第二年7.50，第三年4.67。,300000.0,630.000000,0.002100,630.000000,0.0,630.000000,0,0,1
322664,199.929610,wx1hnn9qf0hm,2005-06-27,2010-06-27,114.171730,40.114990,,04FCB12DCB086712E05400144F67CA43,91515.0,172.970001,...,,91515.0,172.970001,0.001890,172.970001,0.0,172.970001,0,0,1
322665,199.929610,wx1hnn9qf0hm,2005-06-27,2010-06-27,114.171730,40.114990,,04FCB12DCAFC6712E05400144F67CA43,0.0,0.000000,...,,0.0,0.000000,0.003000,0.000000,0.0,0.000000,0,0,1
322666,199.974191,wwg49n01d0k7,2000-11-03,2001-11-03,116.762787,38.441422,,04FCB0F343B36712E05400144F67CA43,15000.0,45.000000,...,,15000.0,45.000000,0.003000,45.000000,0.0,45.000000,0,0,1


In [5]:
def generate_df(insurances, row):
    logging.info(f"generating df for {row['区站号']}")
    insurances["after"]=(insurances["保险起期"]>row["record_date"]).astype(int)
    gdp = client.query_df("select `区站号`,`year`,`province`,`gdp`,`保险密度`,`保险深度` from gdp right join location l on gdp.province=l.`省份`")
    locations = client.query_df("select `保单号`, `区站号` from res")
    df = insurances.merge(locations, on="保单号", how="left")
    df["year"]=df["保险起期"].dt.year
    df = df.merge(gdp, on=["区站号", "year"], how="left")
    return df

df=generate_df(insurances, row)
df

INFO:root:generating df for 54416


,distance,locations,保险起期,保险止期,标的经度,标的纬度,上年保单号,保单号,保险金额,保费合计,...,near,middle,far,after,区站号,year,province,gdp,保险密度,保险深度
0,1.030419,wx4ugd1c433v,2004-07-23,2005-07-23,116.523499,40.221127,,04FCB0F603586712E05400144F67CA43,296000.0,260.000000,...,1,0,0,0,54416,2004,北京,0.204905,441.481878,1.053985
1,1.060964,wx4ug9fpfetr,2005-01-26,2006-01-26,116.523857,40.220924,,04FCB12BDC1A6712E05400144F67CA43,100000.0,49.000000,...,1,0,0,0,54416,2005,北京,0.155193,435.377763,0.936549
2,1.060964,wx4ug9fpfetr,2005-02-02,2006-02-02,116.523857,40.220924,,04FCB12BFC8C6712E05400144F67CA43,60000.0,29.400000,...,1,0,0,0,54416,2005,北京,0.155193,435.377763,0.936549
3,1.060964,wx4ug9fpfetr,2005-02-02,2006-02-02,116.523857,40.220924,,04FCB12BFCEC6712E05400144F67CA43,65000.0,31.850000,...,1,0,0,0,54416,2005,北京,0.155193,435.377763,0.936549
4,1.380379,wx4ugmztp210,2000-09-16,2003-11-16,116.520622,40.242405,,04FCB0F279556712E05400144F67CA43,53190.0,344.670013,...,1,0,0,0,54416,2000,北京,0.180244,235.193605,0.978428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322663,199.894292,wxh2u21jsu9e,2012-05-23,2015-05-22,118.664909,39.507805,,04FCB0F00C7F6712E05400144F67CA43,300000.0,630.000000,...,0,0,1,0,54539,2012,河北,0.083997,356.174952,1.120808
322664,199.929610,wx1hnn9qf0hm,2005-06-27,2010-06-27,114.171730,40.114990,,04FCB12DCB086712E05400144F67CA43,91515.0,172.970001,...,0,0,1,0,53490,2005,山西,0.184568,94.108267,0.774022
322665,199.929610,wx1hnn9qf0hm,2005-06-27,2010-06-27,114.171730,40.114990,,04FCB12DCAFC6712E05400144F67CA43,0.0,0.000000,...,0,0,1,0,53490,2005,山西,0.184568,94.108267,0.774022
322666,199.974191,wwg49n01d0k7,2000-11-03,2001-11-03,116.762787,38.441422,,04FCB0F343B36712E05400144F67CA43,15000.0,45.000000,...,0,0,1,0,54624,2000,河北,0.117357,32.525622,0.469034


In [6]:
def parse_data(row):
    insurances=fetch_insurances(row)
    rainings=fetch_extra_rainings(row)
    insurances=delete_extras(rainings, insurances)
    df=generate_df(insurances, row)
    return df, row["区站号"]

res = []
for _, row in events.iterrows():
    df=parse_data(row)
    res.append(df)
[i[0].shape for i in res]

INFO:root:fetching insurances for 58730
INFO:root:fetching extra rainings for 58730
INFO:root:no extra rainings
INFO:root:generating df for 58730
INFO:root:fetching insurances for 54416
INFO:root:fetching extra rainings for 54416
INFO:root:no extra rainings
INFO:root:generating df for 54416


[(85621, 44), (322668, 44)]

In [10]:
def parse_df(df):
    df=df.copy()
    df["t"]=df["保险起期"].dt.year
    df["历史投保"] = df["上年保单号"].map(lambda x: 1 if x else 0)
    df = df[(df["t"] > 2005) & (df["t"] < 2014)]
    df["ti"] = (df["t"]-2000).astype(str)

    df.rename(
    columns={
        "保费合计": "Premium",
        "保险金额": "Coverage",
        "middle": "Neighbor",
        "near": "Disaster",
        "after": "Post",
        "历史投保": "Prem_before",
        "保险财产购置价": "Price",
        "建筑面积": "Area",
        "保险密度": "Density",
        "保险深度": "Penetration",
        "gdp": "GDP"
    },
    inplace=True,
)
    df["Price"]=df["Price"]/10000
    columns = ['Premium', 'Coverage', 'Price', 'Density', 'Penetration', 'GDP']
    minmax=[0.005,0.995]
    quantiles=df[columns].quantile(minmax)
    for column in columns:
        df=df[(df[column]>quantiles.loc[minmax[0], column]) & (df[column]<quantiles.loc[minmax[1], column])]
    df=df[df["Price"]>0]
    return df
result = []
header=[]
for df,station in res:
    logging.info(f"processing {station}")
    df = parse_df(df)
    model1 = smf.ols("log(Coverage) ~ Neighbor*Post+Prem_before+log(Price)+log(Penetration)+log(GDP)+C(ti)", data=df[(df["Disaster"]==0)]).fit()
    model2 = smf.ols("log(Coverage) ~ Disaster*Post+Prem_before+log(Price)+log(Penetration)+log(GDP)+C(ti)", data=df[(df["Neighbor"]==0)]).fit()
    header.append(str(station))
    header.append(str(station))
    result.append(model1)
    result.append(model2)
stars=Stargazer(result)
stars.custom_columns(header)
stars

INFO:root:processing 58730
INFO:root:processing 54416


In [12]:
with open("output.tex", "w") as f:
    f.write(stars.render_latex())